<a href="https://colab.research.google.com/github/TomonoriSatoh/ai-basics/blob/main/230909_simple_chat_with_simple_context_7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI APIを利用した、基礎的な AI Chat
ユーザーが農機具の故障に関する質問をするたびに、AIが回答を生成し、サポートコンテキストを保持することができる対話型のシステム



In [ ]:
# 前提：事前に、ＯｐｅｎAI API キーを作成し、secret manager に保存しておく
! pip install google-cloud-secret-manager

from google.colab import auth
auth.authenticate_user()

In [2]:
from google.cloud import secretmanager
def access_secret(project_id, secret_name, version='latest'):
    client = secretmanager.SecretManagerServiceClient()
    name = client.secret_version_path(project_id, secret_name, version)
    response = client.access_secret_version(request={"name":name})
    payload = response.payload.data.decode("UTF-8")
    return payload

# 以下、secret manager が属す PROJECT_ID と、事前作成したSECRET_NAMEを指定
PROJECT_ID = "public-but-beclouded"
SECRET_NAME  = "open-ai-api-secret-01"

In [ ]:
! pip install openai
import openai
openai.api_key = access_secret(PROJECT_ID, SECRET_NAME)

以下、[gihyo-ChatGPT, 7_2](https://github.com/gamasenninn/gihyo-ChatGPT/blob/main/notebooks/7_2_chatbot_summary.ipynb)　をベースにカスタマイズ



In [4]:
from json.decoder import JSONDecodeError
import os
import re
import json
import time

In [ ]:
DEBUG = True
#DEBUG = False

# Predefined system message
SYSTEM_MESSAGE = """

下記の前提知識とサポート記録を踏まえてサポートをしてください。
すべての会話の返答とその要約を次のJSON形式で出力してください

## 出力（JSON形式）
{{"content" : "(返答の内容)", "summary" : "(返答を要約した内容)" }}

## 会話の例
USER->機械の調子が悪いです
AI->{{"content" : "具体的に教えてください。どのような機械で、どんな不具合が起こっていますか？","summary" : "機械の種類と不具合の現象を、具体的にユーザに確認。"}}

USER->ありがとう。(終了の暗示や挨拶や感謝の言葉なども含む)
AI->{{"content" : "どういたしまして。いつでもご相談ください。","summary" : "サポート終了。"}}

## 前提知識
  - あなた（AI）は農機具故障診断のエキスパートです。
  - ユーザ(USER)の発言と同じ文言を、そのままユーザに返しては絶対に駄目です。

## サポート記録
  {support_context}

"""

def create_chat_response(user_text, support_context):
  for i in range(3):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": SYSTEM_MESSAGE.format(support_context=support_context)
            },
            {
                "role": "user",
                "content": f"JSON形式で出力してください:[{user_text}]+"
            }
        ],
    )
    try:
      # レスポンスからJSON形式のデータだけを抜き取る
      # Open AI APIリファレンス: https://platform.openai.com/docs/api-reference/chat/streaming
      json_str = re.search(r'\{.*\}', response.choices[0].message['content']).group(0)
      return json.loads(json_str)
    except JSONDecodeError:
      #不正なJSONデータの場合リトライする。そのとき、AIに注意を促す。
      if DEBUG: print("JSON Parse Error:",response.choices[0].message['content'])
      print("...AIの出力でエラーが発生しました。リトライします......")
      user_text += '...JSON形式のレスポンスを守ってください。'
      time.sleep(1)
      continue
    except AttributeError:
      #JSON変換できなかったもの。リストなどの場合。そのままの値をセットする。
      if DEBUG: print("Attribute Error:",response.choices[0].message['content'])
      return {
          "content": response.choices[0].message['content'],
          "summary": response.choices[0].message['content'],
          "error": "attribute error"
      }

  else:
    return []

support_context = ""
print("農機具についてお困りがあればお答えします\nなんでも聞いてください。\n")

while True:
  user_text = input("\n>")

  if user_text == "quit":
    break

  response = create_chat_response(user_text, support_context)
  if response:
      answer  = response['content']
      summary = response['summary']
  else:
    print("申し訳ございません。質問の方法を変えてください。")
    continue

  support_context += f"- user: {user_text}\n"
  support_context += f"- AI: {summary}\n\n"
  print("\n返答:",answer)

  if DEBUG:
    print("要約:",summary)
    print("response:",response)
